# Postcleaning code

To be run once Business filled the required missing data. 

We will be using inner join and merge only the PC data leaving the rest of the data. One8, Coty brands are not required for analysis.

In [84]:
import pandas as pd
import numpy as np

In [85]:
from datetime import date

In [86]:
import datetime

In [87]:
pd.options.mode.chained_assignment = None #for not showing warning message 
pd.options.display.float_format = '{:,.7f}'.format #for setting thhe float format to this notebook

In [88]:
# jccycle                   =      pd.read_excel("D:\Analytics\Marketing Report\Journey Cycle\Master Journey Cycle.xlsx")
# product_hierarchy         =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Product Hierarchy.xlsx",
#                                                sheet_name= "Material Master Data")
# static_category_lookup    =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Static_Category_Lookup.xlsx")
# statename_lookup          =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Statename.xlsx")

In [89]:
# data9_keyac = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-Key Ac's.xlsb",engine='pyxlsb')
# data9_nonsouth = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-Non South.xlsb",engine = 'pyxlsb')
# data9_south = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-South.xlsb",engine = 'pyxlsb')

In [90]:
data9 =  data9_keyac.append(data9_nonsouth)
data9 = data9.append(data9_south)

In [91]:
data10 = data9

In [92]:
temp4 = data9.isnull().sum().to_frame().reset_index()
temp4[temp4[0] != 0]

,index,0
2,DistName,8052
3,DistType,8052
4,Rtrid,8052
5,RtrCode,11094
6,RtrName,11094
7,RtrGroupCde,11094
9,BillNo,8053
10,InvDate,8052
13,PrdName,8052
16,NetAmt,8052


In [93]:
data9.drop(["PrdName"],axis = 1,inplace = True)
data9.drop(["BaseValue"],axis = 1,inplace = True)

In [94]:
data9.iloc[1]

Distcode              MT0600032000290003200029
RS CODE                                3200029
DistName         VENKATA SAI SALES CORPORATION
DistType                   Key A/c Distributor
Rtrid                               48.0000000
RtrCode                                RET0048
RtrName        AVENUE SUPERMARTS LTD RAMANTPUR
RtrGroupCde             AVENUE SUPER MARTS LTD
RET GRP                           Modern Trade
BillNo                              BIL1900317
InvDate                         43,776.0000000
PERIOD                                    WK01
PrdCode                         FCAM0050NAT03R
BRAND CAT                    FAIREVER  Natural
TotQty                                      96
NetAmt                           6,719.7700000
In Lakhs                             0.0598082
REGION                               Keys-Ac/s
ZONE                                 Keys-Ac/s
Name: 1, dtype: object

In [95]:
data9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126587 entries, 0 to 740834
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Distcode     1126587 non-null  object 
 1   RS CODE      1126587 non-null  int64  
 2   DistName     1118535 non-null  object 
 3   DistType     1118535 non-null  object 
 4   Rtrid        1118535 non-null  float64
 5   RtrCode      1115493 non-null  object 
 6   RtrName      1115493 non-null  object 
 7   RtrGroupCde  1115493 non-null  object 
 8   RET GRP      1126587 non-null  object 
 9   BillNo       1118534 non-null  object 
 10  InvDate      1118535 non-null  float64
 11  PERIOD       1126587 non-null  object 
 12  PrdCode      1126587 non-null  object 
 13  BRAND CAT    1126587 non-null  object 
 14  TotQty       1126587 non-null  int64  
 15  NetAmt       1118535 non-null  float64
 16  In Lakhs     1126587 non-null  float64
 17  REGION       1126587 non-null  object 
 18  ZON

In [96]:
data9.columns

Index(['Distcode', 'RS CODE', 'DistName', 'DistType', 'Rtrid', 'RtrCode',
       'RtrName', 'RtrGroupCde', 'RET GRP', 'BillNo', 'InvDate', 'PERIOD',
       'PrdCode', 'BRAND CAT', 'TotQty', 'NetAmt', 'In Lakhs', 'REGION',
       'ZONE'],
      dtype='object')

In [97]:
data9.shape

(1126587, 19)

In [98]:
data9.columns       = map(str.lower, data9.columns)

In [99]:
data9 = data9.rename(columns = {"distname"    : "cusname",
                                "disttype"    : "cusgroname",
                                "rtrgroupcde" : "category",
                                "brand pack"  : "brandpack",
                                "in lakhs"    : "basevalue"
                               })

In [100]:
data9[["rtrcode","ret grp","category","zone"]] = data9[["rtrcode","ret grp","category","zone"]].applymap(lambda s:(s.upper() if type(s)==str else s))

In [101]:
data9 = pd.merge(data9, product_hierarchy, 
                 left_on = 'prdcode', 
                 right_on = 'PrdCode',
                 how='inner')
data9.drop(columns  = ["PrdCode"] ,axis = 1,inplace = True)

In [102]:
data9.columns       = map(str.lower, data9.columns)

In [103]:
data9[data9["category"].isna()]["ret grp"].unique()

array(['E-COMM', 'MODERN TRADE', 'PC-KIRANA/GENERAL STORES'], dtype=object)

In [104]:
data9["category"]   = data9["category"].fillna(value = data9["ret grp"])

In [105]:
category_check = pd.merge(data9, static_category_lookup,how='left',left_on = "category",right_on="Old_Category")

In [106]:
category_check[category_check["Old_Category"].isna()][["ret grp","Old_Category"]]

,ret grp,Old_Category


In [107]:
data9[data9["brand pack"].isna()]["prdcode"]

Series([], Name: prdcode, dtype: object)

In [108]:
data9[data9["cusname"].isna()]["ret grp"].unique()

array(['E-COMM', 'MODERN TRADE', 'PC-KIRANA/GENERAL STORES'], dtype=object)

In [109]:
data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "cusname"]    = data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "cusname"].fillna("Kirana")
data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "cusgroname"] = data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "cusgroname"].fillna("Kirana")
data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrid"]      = data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrid"].fillna("Kirana")
data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrcode"]    = data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrcode"].fillna("Kirana")
data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrname"]    = data9.loc[data9["ret grp"] == "PC-KIRANA/GENERAL STORES", "rtrname"].fillna("Kirana")
data9.loc[data9["ret grp"] == "MODERN TRADE", "cusname"]                = data9.loc[data9["ret grp"] == "MODERN TRADE", "cusname"].fillna("Modern Trade")
data9.loc[data9["ret grp"] == "MODERN TRADE", "cusgroname"]             = data9.loc[data9["ret grp"] == "MODERN TRADE", "cusgroname"].fillna("Modern Trade")
data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrid"]                  = data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrid"].fillna("Modern Trade")
data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrcode"]                = data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrcode"].fillna("Modern Trade")
data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrname"]                = data9.loc[data9["ret grp"] == "MODERN TRADE", "rtrname"].fillna("Modern Trade")
data9.loc[data9["ret grp"] == "E-COMM", "cusname"]                      = data9.loc[data9["ret grp"] == "E-COMM", "cusname"].fillna("E-Commerce")
data9.loc[data9["ret grp"] == "E-COMM", "cusgroname"]                   = data9.loc[data9["ret grp"] == "E-COMM", "cusgroname"].fillna("E-Commerce")
data9.loc[data9["ret grp"] == "E-COMM", "rtrid"]                        = data9.loc[data9["ret grp"] == "E-COMM", "rtrid"].fillna("E-Commerce")
data9.loc[data9["ret grp"] == "E-COMM", "rtrcode"]                      = data9.loc[data9["ret grp"] == "E-COMM", "rtrcode"].fillna("E-Commerce")
data9.loc[data9["ret grp"] == "E-COMM", "rtrname"]                      = data9.loc[data9["ret grp"] == "E-COMM", "rtrname"].fillna("E-Commerce")


In [110]:
data9["billno"] = data9["billno"].fillna(value = 0)
data9["netamt"] = data9["netamt"].fillna(value = 0)

In [111]:
data9["invdate"] = data9["invdate"].fillna(data9["invdate"].mode()[0])

In [112]:
data9["invdate"] = data9["invdate"].astype("int")

In [113]:
data9["invdate"].unique()

array([43776, 43777, 43778, 43780, 43781, 43782, 43783, 43784, 43785,
       43786, 43787, 43788, 43789, 43790, 43791, 43792, 43794, 43795,
       43796, 43797, 43798, 43799, 43800, 43801, 43802, 43803, 43793,
       43779])

In [114]:
from datetime import datetime, timedelta

def xldate_to_datetime(xldate):
   tempDate = datetime(1900, 1, 1)
   deltaDays =timedelta(days=int(xldate)-2)
   TheTime = (tempDate + deltaDays )
   return TheTime.strftime("%Y/%m/%d")

In [115]:
data9["invdate"] = data9["invdate"].apply(lambda x: xldate_to_datetime(x))

In [116]:
data9["invdate"].unique()

array(['2019/11/07', '2019/11/08', '2019/11/09', '2019/11/11',
       '2019/11/12', '2019/11/13', '2019/11/14', '2019/11/15',
       '2019/11/16', '2019/11/17', '2019/11/18', '2019/11/19',
       '2019/11/20', '2019/11/21', '2019/11/22', '2019/11/23',
       '2019/11/25', '2019/11/26', '2019/11/27', '2019/11/28',
       '2019/11/29', '2019/11/30', '2019/12/01', '2019/12/02',
       '2019/12/03', '2019/12/04', '2019/11/24', '2019/11/10'],
      dtype=object)

In [117]:
data9["invdate"] = pd.to_datetime(data9["invdate"])

In [118]:
data9               = pd.merge(data9, jccycle, left_on = "invdate", right_on = "Wk_Start_Date")
data9.drop(columns  = ["Wk_Start_Date"] ,axis = 1,inplace = True)

In [119]:
data9.columns

Index(['distcode', 'rs code', 'cusname', 'cusgroname', 'rtrid', 'rtrcode',
       'rtrname', 'category', 'ret grp', 'billno', 'invdate', 'period',
       'prdcode', 'brand cat', 'totqty', 'netamt', 'basevalue', 'region',
       'zone', 'prdname', 'brand', 'retail brand', 'brand pack', 'div', 'Jc',
       'Week'],
      dtype='object')

In [120]:
data9               = pd.merge(data9, statename_lookup, on = "zone", how = "left" )

In [121]:
data9 = data9.rename(columns = {"distname"    : "cusname",
                                "disttype"    : "cusgroname",
                                "rtrgroupcde" : "category",
                                "brand pack"  : "brandpack",
                                "in lakhs"    : "basevalue"
                               })

In [128]:
data = data9[["statename","distcode","cusname","cusgroname","rtrid",
               "rtrcode","rtrname","invdate","brand","brandpack",
               "totqty","netamt","basevalue","category","prdcode",
               "div","Jc","Week"]]

In [129]:
test = data.head(1)

In [130]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   statename   1 non-null      object        
 1   distcode    1 non-null      object        
 2   cusname     1 non-null      object        
 3   cusgroname  1 non-null      object        
 4   rtrid       1 non-null      object        
 5   rtrcode     1 non-null      object        
 6   rtrname     1 non-null      object        
 7   invdate     1 non-null      datetime64[ns]
 8   brand       1 non-null      object        
 9   brandpack   1 non-null      object        
 10  totqty      1 non-null      int64         
 11  netamt      1 non-null      float64       
 12  basevalue   1 non-null      float64       
 13  category    1 non-null      object        
 14  prdcode     1 non-null      object        
 15  div         1 non-null      object        
 16  Jc          1 non-null      ob

In [131]:
test = pd.DataFrame(test)

In [132]:
data.shape

(1104822, 18)

# SQL CONNECTION

In [137]:
import pyodbc 

In [138]:
connStr = ("DRIVER={ODBC Driver 13 for SQL Server};"
           "Server=DESKTOP-KGNPIA3\SQLEXPRESS;"
           "Database=sso_db;"
           "Trusted_Connection=yes")
conn = pyodbc.connect(connStr)

In [139]:
cursor = conn.cursor()
for index,row in data.iterrows():
    cursor.execute("INSERT INTO new_cleaned_data(statename,distcode,cusname,cusgroname,rtrid,rtrcode,rtrname,invdate,brand,prdcode,brandpack,totqty,netamt,basevalue,category,div,Jc,Week) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);",
                        row['statename'],
                        row['distcode'],
                        row['cusname'],
                        row['cusgroname'],
                        row['rtrid'],
                        row['rtrcode'],
                        row['rtrname'],
                        row['invdate'],
                        row['brand'],
                        row['prdcode'],
                        row['brandpack'],
                        row['totqty'],
                        row['netamt'],
                        row['basevalue'],
                        row['category'],
                        row['div'],
                        row['Jc'],
                        row['Week'])

    conn.commit()
cursor.close()
conn.close()